---
title: "Level of readiness of the bioeconomy in Colombia"
description: "Analysis of the capacities of departments to develop the bioeconomy in Colombia at the departmental level"
categories:
  - Bioeconomy
  - Departments
  - D3
  - Humboldt Institute
date: "2023-04-20"
image: "https://github.com/jadeynryan/orcas/blob/master/inst/img/leaflet_screenshot.png?raw=true"
toc: true
---

```{ojs}
d3 = require("d3@7")
navio_npm = require("navio@0.0.75")
Inputs = require("@observablehq/inputs")
htl = require("htl")

```

```{ojs}
aq = {
  const aq = await require(`arquero@${aq_version}`);

  // Add HTML table view method to tables
  Object.assign(aq.ColumnTable.prototype, {
    view(options) { return toView(this, options); }
  });

  return aq;
}
```

```{ojs}
aq_version = '7.2.0'
```

```{ojs}
op = aq.op
```

```{ojs}
toView = {
  const DEFAULT_LIMIT = 100;
  const DEFAULT_NULL = value => `<span style="color: #999;">${value}</span>`;
  const tableStyle = 'margin: 0; border-collapse: separate; border-spacing: 0; width: initial;';
  const cellStyle = 'padding: 1px 5px; white-space: nowrap; overflow-x: hidden; text-overflow: ellipsis; font-variant-numeric: tabular-nums;';
  const rowStyle = 'border-bottom: 1px solid #eee;';

  // given an Arquero data table, provide an HTML table view
  return function(dt, opt = {}) {
    // permit shorthand for limit
    if (typeof opt === 'number') opt = { limit: opt };
    
    // marshal cell color options
    const color = { ...opt.color };
    if (typeof opt.color === 'function') {
      // if function, apply to all columns
      dt.columnNames().forEach(name => color[name] = opt.color);
    } else {
      // otherwise, gather per-column color options
      for (const key in color) {
        const value = color[key];
        color[key] = typeof value === 'function' ? value : () => value;
      }
    }

    // marshal CSS styles as toHTML() options
    const table = `${tableStyle}`;
    const cell = (name, index, row, th) => {
      return `${cellStyle} max-width: ${+opt.maxCellWidth || 300}px;`
        + ` border-bottom: solid 1px ${th ? '#ccc' : '#eee'};`
        + (color[name] ? ` background-color: ${color[name](index, row)};` : '');
    };
    const td = (name, index, row) => cell(name, index, row, false);
    const th = (name, index, row) => `position: sticky; top: 0; background: #fff; `
        + cell(name, index, row, true);

    opt = {
      limit: DEFAULT_LIMIT,
      null: DEFAULT_NULL,
      ...opt,
      style: { table, td, th }
    };

    // return container div, bind table value to support viewof operator
    const size = `max-height: ${+opt.height || 270}px`;
    const style = `${size}; overflow-x: auto; overflow-y: auto;`;
    const view = html`<div style="${style}">${dt.toHTML(opt)}</div>`;
    return Object.assign(view, { value: dt });
  };
}
```

```{ojs}
function Scrubber(values, {
  format = value => value,
  initial = 0,
  direction = 1,
  delay = null,
  autoplay = true,
  loop = true,
  loopDelay = null,
  alternate = false
} = {}) {
  values = Array.from(values);
  const form = html`<form style="font: 12px var(--sans-serif); font-variant-numeric: tabular-nums; display: flex; height: 33px; align-items: center;">
  <button name=b type=button style="margin-right: 0.4em; width: 5em;"></button>
  <label style="display: flex; align-items: center;">
    <input name=i type=range min=0 max=${values.length - 1} value=${initial} step=1 style="width: 180px;">
    <output name=o style="margin-left: 0.4em;"></output>
  </label>
</form>`;
  let frame = null;
  let timer = null;
  let interval = null;
  function start() {
    form.b.textContent = "Pause";
    if (delay === null) frame = requestAnimationFrame(tick);
    else interval = setInterval(tick, delay);
  }
  function stop() {
    form.b.textContent = "Play";
    if (frame !== null) cancelAnimationFrame(frame), frame = null;
    if (timer !== null) clearTimeout(timer), timer = null;
    if (interval !== null) clearInterval(interval), interval = null;
  }
  function running() {
    return frame !== null || timer !== null || interval !== null;
  }
  function tick() {
    if (form.i.valueAsNumber === (direction > 0 ? values.length - 1 : direction < 0 ? 0 : NaN)) {
      if (!loop) return stop();
      if (alternate) direction = -direction;
      if (loopDelay !== null) {
        if (frame !== null) cancelAnimationFrame(frame), frame = null;
        if (interval !== null) clearInterval(interval), interval = null;
        timer = setTimeout(() => (step(), start()), loopDelay);
        return;
      }
    }
    if (delay === null) frame = requestAnimationFrame(tick);
    step();
  }
  function step() {
    form.i.valueAsNumber = (form.i.valueAsNumber + direction + values.length) % values.length;
    form.i.dispatchEvent(new CustomEvent("input", {bubbles: true}));
  }
  form.i.oninput = event => {
    if (event && event.isTrusted && running()) stop();
    form.value = values[form.i.valueAsNumber];
    form.o.value = format(form.value, form.i.valueAsNumber, values);
  };
  form.b.onclick = () => {
    if (running()) return stop();
    direction = alternate && form.i.valueAsNumber === values.length - 1 ? -1 : 1;
    form.i.valueAsNumber = (form.i.valueAsNumber + direction) % values.length;
    form.i.dispatchEvent(new CustomEvent("input", {bubbles: true}));
    start();
  };
  form.i.oninput();
  if (autoplay) start();
  else stop();
  Inputs.disposal(form).then(stop);
  return form;
}
```

```{ojs}
async function navio(data, _options = {}) {
  const options = {
    height: 300, // Navio's height
    attribs: null, // array of attrib names to be used, leave as null for all of them

    x0: 0, //Where to start drawing navio in x
    y0: 100, //Where to start drawing navio in y, useful if your attrib names are too long
    maxNumDistictForCategorical: 10, // addAllAttribs uses this for deciding if an attribute is categorical (has less than     maxNumDistictForCategorical categories) or ordered
    maxNumDistictForOrdered: 90, // addAllAttribs uses this for deciding if an attribute is ordered (has less than     maxNumDistictForCategorical categories) or text. Use    maxNumDistictForOrdered : Infinity for never choosing Text

    howManyItemsShouldSearchForNotNull: 100, // How many rows should addAllAttribs search to decide guess an attribute type
    margin: 10, // Margin around navio

    levelsSeparation: 40, // Separation between the levels
    divisionsColor: "white", // Border color for the divisions
    levelConnectionsColor: "rgba(205, 220, 163, 0.5)", // Color for the conections between levels
    divisionsThreshold: 4, // What's the minimum row height needed to draw divisions
    fmtCounts: d3.format(",.0d"), // Format used to display the counts on the bottom
    legendFont: "14px sans-serif", // The font for the header
    nestedFilters: true, // Should navio use nested levels?

    showAttribTitles: true, // Show headers?
    attribWidth: 15, // Width of the columns
    attribRotation: -45, // Headers rotation
    attribFontSize: 13, // Headers font size
    attribFontSizeSelected: 32, // Headers font size when mouse over

    filterFontSize: 10, // Font size of the filters explanations on the bottom

    tooltipFontSize: 12, // Font size for the tooltip
    tooltipBgColor: "#b2ddf1", // Font color for tooltip background
    tooltipMargin: 50, // How much to separate the tooltip from the cursor
    tooltipArrowSize: 10, // How big is the arrow on the tooltip

    digitsForText: 2, // How many digits to use for text attributes

    addAllAttribsRecursionLevel: Infinity, // How many levels depth do we keep on adding nested attributes
    addAllAttribsIncludeObjects: true, // Should addAllAttribs include objects
    addAllAttribsIncludeArrays: true, // Should addAllAttribs include arrays

    nullColor: "#ffedfd", // Color for null values
    defaultColorInterpolator: d3.interpolateBlues,
    defaultColorInterpolatorDate: d3.interpolatePurples,
    defaultColorInterpolatorDiverging: d3.interpolateBrBG,
    defaultColorInterpolatorOrdered: d3.interpolateOranges,
    defaultColorInterpolatorText: d3.interpolateGreys,
    defaultColorRangeBoolean: ["#a1d76a", "#e9a3c9", "white"], //true false null
    defaultColorRangeSelected: ["white", "#b5cf6b"],
    defaultColorCategorical: d3.schemeCategory10,

    showSelectedAttrib: true, // Display the attribute that shows if a row is selected
    showSequenceIDAttrib: true, // Display the attribute with the sequence ID

    ..._options
  };

  let div = html`<div  style="display:block; overflow-x:scroll"></div>`;
  
  // Create the navio
  const nv = navio_npm(d3.select(div), options.height);

  for (let opt in options) {
    if (opt === "id") {
      nv.id(options[opt]);
    } else if (opt !== "attribs") {
      nv[opt] = options[opt];
    }
  }

  // Add the data
  nv.data(data);

  if (options.attribs) {
    nv.addAllAttribs(options.attribs);
  } else {
    nv.addAllAttribs();
  }

  nv.updateCallback(() => {
    div.value = nv.getVisible();
    div.dispatchEvent(new Event("input", { bubbles: true }));
    // notify(div);
  });

  div.value = data;
  div.nv = nv;
  return div;
}
```

```{ojs}
data_hubs0 = FileAttachment("Hubs_bioec_col.csv").csv({ typed: true })
data_hubs = data_hubs0.map(d => ({ ...d, name: d.plataforma, type: "Bioeconomy" }));
```

```{ojs}
dData = new Map(data_hubs.map(d => [d.name, d]))
```

```{ojs}
hullColor = d3.scaleOrdinal(
  d3.quantize(
    colorInterpolator,
    tidyData.groupby(groupBy).count().objects().length
  )
)
```

```{ojs}
dConnections = new Map()
```

```{ojs}
graph = {
  for (let n of dData.values()) {
    n.degree = 0;
  }

  const linksArray = links.objects();
  for (let l of linksArray) {
    const source = dData.get(l.source);
    // target = dNodes.get(l.target);

    source.cluster = l.target;
    // target.cluster = l.target;
    source.degree += 1;
    // target.degree += 1;
  }

  const getOrCreateNode = (name) => {
    let node = dConnections.get(name);

    if (!node) {
      node = {
        name,
        type: "Connection",
        cluster: name,
        x: width / 2 + (Math.random() * width) / 10,
        y: fheight_node.height / 2 + (Math.random() * fheight_node.height) / 10
      };
      dConnections.set(name, node);
    }
    return node;
  };

  // --------- Filter Links and Nodes --------------
  const filteredLinks = linksArray.map((l) => {
    return {
      source: dData.get(l.source),
      target: getOrCreateNode(l.target)
    };
  });

  const filteredNodesObject = filteredLinks.reduce(
    (p, l) => {
      if (!p.set.has(l.source.name)) {
        p.set.add(l.source.name);
        p.list.push(l.source);
      }
      if (!p.set.has(l.target.name)) {
        p.set.add(l.target.name);
        p.list.push(l.target);
      }

      return p;
    },
    { set: new Set(), list: [] }
  );

  // Clusters is used for the hulls
  const clusters = d3.rollups(
    filteredLinks,
    (v) => v.map((l) => l.source).concat([v[0].target]),
    (d) => d.target.name
  );

  return {
    nodes: filteredNodesObject.list,
    links: linksArray,
    clusters: clusters
  };
}
```

```{ojs}
dateFmt = d3.timeParse("%m/%d/%y")
```

```{ojs}
tidyData = aq
  .from(data_hubs)
  .derive({
    "linea_investigacion": (d) =>
      d["linea_investigacion"] === null ? [null] : op.split(d["linea_investigacion"], ";"),
    "departamento": (d) =>
      d["departamento"] === null ? [null] : op.split(d["departamento"], ";")
  })
  .unroll("linea_investigacion")
  .unroll("departamento")
```

```{ojs}
links = tidyData
  .filter(aq.escape((d) => !d.ano_creacion || d.ano_creacion <= minDate)) // Filter by Date
  .derive({
    source: (d) => d.name,
    target: aq.escape((d) => d[groupBy])
  })
  .groupby(["source", "target"])
  .count()
  .filter((d) => d.target && d.source) // Ignore nulls
  .filter(aq.escape((d) => selectedTargets.includes(d.target))) // Filter by selectedTarget
  .select(["source", "target"])
```

```{ojs}
targets = links
  .groupby("target")
  .count()
  .rename({ target: "name" })
```

```{ojs}
sources = links
  .groupby("source")
  .count()
  .rename({ source: "name" })
  .derive({ type: () => "Bioeconomy" })
```

```{ojs}
col_names = Object.keys(data_hubs[0])
```

```{ojs}
template = (inputs) => 
htl.html`<div class="styled">${Object.values(inputs)}</div>
<style>
  div.styled {
    text-align: left;
    column-count: 2
  }
  div.styled label {
    font-weight: bold;
    line-height: 200%;
  }
  div.styled label:not(div>label):after {
    content: ":";
  }
</style>`
```

```{ojs}
viewof minDate = {
  const dates = data_hubs
    .map((d) => d.ano_creacion)
    .filter((d) => d)
    .sort((a, b) => d3.ascending(+a, +b));
  return Scrubber(dates, {
    delay: 500,
    autoplay: false,
    loop:false,
    initial: dates.length-1
  });
}
```

```{ojs}
viewof groupBy = {
  // `["Last Name","First Name","Appointments","Job Profile","Picture URL","Research Areas","Labs and Groups","Institutes and Centers","Date of joining NU","Courses","Campus","Supervisory Organization","Title","Academic Track Type","Degrees","Last School Attended","Year Highest Degree Received","Interdisciplinary Dept 1","Interdisciplinary College 1","Interdisciplinary Dept 2","Interdisciplinary College 2","Interdisciplinary Dept 3","Interdisciplinary College 3","name","selected","__seqId","__i","Appointment"]`

  const attrs = [
  //"plataforma",
  //"descripcion",
  //"link",
  "tematica",
  "fases",
  "relacion",
  "linea_investigacion",
  //"partners",
  //"country_host",
  ////"ano_creacion",
  //"departamentos",
  //"def_bioeconomia"
  //"relaciones"
  //"logo"
  //"logo_viz"
  //"lat_host"
  //"long_host"
  ];
  return Inputs.select(attrs, { label: "Agrupación", value: "fases" });
}
```

```{ojs}
viewof selectedTargets = {
  const targetNames = tidyData
    .groupby(groupBy)
    .count()
    .objects()
    .map((d) => d[groupBy])
    .filter((d) => d);
  const selection =
    groupBy === "linea_investigacion"
      ? targetNames.filter((d) => !["Bioindustría","Bioenergía","Forestal"].includes(d))
      : targetNames;
  const cr = 15;
  return Inputs.checkbox(targetNames, {
    label: "Show",
    value: selection,
    format: (d) => html`<div style="height: ${cr}px;
    line-height: ${cr}px; 
    display: flex;
    align-content: center;
    flex-wrap: wrap;
"><svg width=${cr} height=${cr} style="margin-right:5px">
<circle r=${cr / 2} cx=${cr / 2} cy=${cr / 2} fill="${hullColor(d)}"/> 
</svg>
${d}</div>`
  });
}
```

```{ojs}
viewof fheight_node = (
  Inputs.form({
  height: Inputs.range([200, 1400], {label: "Height", step:1, value: 420}),
  nodeSize: Inputs.range([2, 30], {label: "Node Size",value: 23,step: 1})},
          
   // borrowed template from Allison Horst - splits up in two columns
      {template})
  )
```




::: {#chart-container}
:::




```{js}
chart = {
  const container = document.querySelector("#chart-container");  // Selecciona el contenedor explícitamente
  container.innerHTML = "";  // Limpia el contenedor antes de agregar el SVG

  const svg = d3.create("svg")
    .attr("width", width)
    .attr("height", fheight_node.height)
    .attr("viewBox", [0, 0, width, fheight_node.height])
    .attr(
      "style",
      "max-width: 100%; height: auto; height: intrinsic; overflow: visible"
    );

  // Resto del código de tu gráfico aquí...
  // Agrega el SVG al contenedor seleccionado
  container.appendChild(svg.node());

  return container;
};

```

```{ojs}
viewof colorInterpolator = {
  const colorSchemes = [
    "interpolateBlues",
    "interpolateBrBG",
    "interpolateBuGn",
    "interpolateBuPu",
    "interpolateCividis",
    "interpolateCool",
    "interpolateCubehelixDefault",
    "interpolateGnBu",
    "interpolateGreens",
    "interpolateGreys",
    "interpolateInferno",
    "interpolateMagma",
    "interpolateOrRd",
    "interpolateOranges",
    "interpolatePRGn",
    "interpolatePiYG",
    "interpolatePlasma",
    "interpolatePuBu",
    "interpolatePuBuGn",
    "interpolatePuOr",
    "interpolatePuRd",
    "interpolatePurples",
    "interpolateRainbow",
    "interpolateRdBu",
    "interpolateRdGy",
    "interpolateRdPu",
    "interpolateRdYlBu",
    "interpolateRdYlGn",
    "interpolateReds",
    "interpolateSinebow",
    "interpolateSpectral",
    "interpolateString",
    "interpolateTurbo",
    "interpolateViridis",
    "interpolateWarm",
    "interpolateYlGn",
    "interpolateYlGnBu",
    "interpolateYlOrBr",
    "interpolateYlOrRd"
  ];
  const colors = new Map(
    [["Custom Turbo", (t) => d3.interpolateTurbo(0.2 + t * 0.8)]].concat(
      colorSchemes.map((c) => [c, d3[c]])
    )
  );

  return Inputs.select(colors, {
    label: "Color Scheme",
    format: (d) => d[0]
  });
}
```

```{ojs}
viewof ffsize_gnode = (
  Inputs.form({
  fontSize:Inputs.range([1, 36], {label: "Font size", value: 8, step: 1}),
  greyNodeSize:Inputs.range([1, 30], {label: "Connecting Node Size", value: 5, step: 1})},
          
   // borrowed template from Allison Horst - splits up in two columns
      {template})
  )
```

```{ojs}
viewof fimage_gnodes = (
  Inputs.form({
  useImages:Inputs.toggle({label: "Use Images", value: true}),
  showGroupingNodes:Inputs.toggle({label: "Show grey nodes", value: true})},
          
   // borrowed template from Allison Horst - splits up in two columns
      {template})
)
```

```{ojs}
viewof fclabels_tshadows = (
  Inputs.form({
  clipLabels: Inputs.toggle({label: "Clip Labels", value: true}),
  useShadowsText: Inputs.toggle({label: "Use Text Shadows", value: true})},
          
   // borrowed template from Allison Horst - splits up in two columns
      {template})
)
```

```{ojs}
viewof fhpadding_mdlabel = (
  Inputs.form({
  hullPadding: Inputs.range([0, 100], {label: "Hull Padding", value: 9, step: 1}),
  minDegree: Inputs.range([0, d3.max(graph.nodes, (d) => d.degree)+1], {
  label: "Min Degree for Label",
  value: this !== undefined ? this.value : 1,
  step: 1
})},
          
   // borrowed template from Allison Horst - splits up in two columns
      {template})
  )
```

```{ojs}
viewof hullPadding = Inputs.range([0, 100], {label: "Hull Padding", value: 9, step: 1})
```

```{ojs}
viewof minDegree = Inputs.range([0, d3.max(graph.nodes, (d) => d.degree)+1], {
  label: "Min Degree for Label",
  value: this !== undefined ? this.value : 1,
  step: 1
})
```

```{ojs}
viewof useShadowsNodes = Inputs.toggle({label: "Use Node Shadows", value: true})
```

```{ojs}
viewof excludeKhouryHull = Inputs.toggle({label: "Exclude Khoury Hull", value: true})
```